In [1]:
##### reproducibility for Keras
import os
import numpy as np
import random as rn
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
rn.seed(7)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1,
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)

from keras import backend as K

tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

##### Import some librarys
import pickle
import pandas as pd

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/feature_selection_positive.csv')


# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [3]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_pos_fs.sav', 'wb'))

In [4]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_pos_fs.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.29099
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.21112
[3]	valid_0's multi_logloss: 1.14382
[4]	valid_0's multi_logloss: 1.0832
[5]	valid_0's multi_logloss: 1.02917
[6]	valid_0's multi_logloss: 0.982285
[7]	valid_0's multi_logloss: 0.939476
[8]	valid_0's multi_logloss: 0.902268
[9]	valid_0's multi_logloss: 0.869022
[10]	valid_0's multi_logloss: 0.838978
[11]	valid_0's multi_logloss: 0.811207
[12]	valid_0's multi_logloss: 0.784389
[13]	valid_0's multi_logloss: 0.761399
[14]	valid_0's multi_logloss: 0.739735
[15]	valid_0's multi_logloss: 0.720263
[16]	valid_0's multi_logloss: 0.702614
[17]	valid_0's multi_logloss: 0.682459
[18]	valid_0's multi_logloss: 0.667015
[19]	valid_0's multi_logloss: 0.650877
[20]	valid_0's multi_logloss: 0.637697
[21]	valid_0's multi_logloss: 0.625049
[22]	valid_0's multi_logloss: 0.610747
[23]	valid_0's multi_logloss: 0.598023
[24]	valid_0's multi_logloss: 0.586897
[25]	valid_0's mul

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:
# set parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [6]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
pickle.dump(model, open('../model/xgb_pos_fs.sav', 'wb'))
t.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 2s 1ms/step - loss: 1.8700 - acc: 0.2507 - val_loss: 1.4152 - val_acc: 0.3155
Epoch 2/100
1340/1340 [==============================] - 0s 74us/step - loss: 1.7653 - acc: 0.2582 - val_loss: 1.3522 - val_acc: 0.2976
Epoch 3/100
1340/1340 [==============================] - 0s 67us/step - loss: 1.6996 - acc: 0.2813 - val_loss: 1.3382 - val_acc: 0.2887
Epoch 4/100
1340/1340 [==============================] - 0s 69us/step - loss: 1.6141 - acc: 0.2985 - val_loss: 1.3362 - val_acc: 0.2976
Epoch 5/100
1340/1340 [==============================] - 0s 74us/step - loss: 1.5480 - acc: 0.3075 - val_loss: 1.3222 - val_acc: 0.3065
Epoch 6/100
1340/1340 [==============================] - 0s 69us/step - loss: 1.5700 - acc: 0.3299 - val_loss: 1.3119 - val_acc: 0.3214
Epoch 7/100
1340/1340 [==============================] - 0s 67us/step - loss: 1.5446 - acc: 0.3209 - val_loss: 1.2941 - val_acc: 0.4286
Ep

1340/1340 [==============================] - 0s 71us/step - loss: 0.6562 - acc: 0.7090 - val_loss: 0.8641 - val_acc: 0.6845
Epoch 61/100
1340/1340 [==============================] - 0s 89us/step - loss: 0.6338 - acc: 0.7396 - val_loss: 0.8670 - val_acc: 0.6786
Epoch 62/100
1340/1340 [==============================] - 0s 73us/step - loss: 0.6181 - acc: 0.7299 - val_loss: 0.8718 - val_acc: 0.6518
Epoch 63/100
1340/1340 [==============================] - 0s 70us/step - loss: 0.5973 - acc: 0.7418 - val_loss: 0.8769 - val_acc: 0.6548
Epoch 64/100
1340/1340 [==============================] - 0s 75us/step - loss: 0.6177 - acc: 0.7478 - val_loss: 0.8780 - val_acc: 0.6131
Epoch 65/100
1340/1340 [==============================] - 0s 73us/step - loss: 0.5866 - acc: 0.7552 - val_loss: 0.8833 - val_acc: 0.5893
Epoch 66/100
1340/1340 [==============================] - 0s 81us/step - loss: 0.5622 - acc: 0.7701 - val_loss: 0.8912 - val_acc: 0.5685
Epoch 67/100
1340/1340 [==============================

In [8]:
f = []
f.append(t)

In [9]:
f

[[71.43, 84.52, 81.25, 61.61]]

In [10]:
#read data
df = pd.read_csv('../data/feature_selection_negative.csv')

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [11]:
# define and fit 
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
print(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_ng_fs.sav', 'wb'))

70.21


In [12]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

# pickle.dump(gbm, open('../model/lgbm_ng_fs.sav', 'wb'))
# result append to list
t.append(
round(gbm.score(X_test, y_test)*100, 2)
)
pickle.dump(gbm, open('../model/lgbm_ng_fs.sav', 'wb'))

[1]	valid_0's multi_logloss: 1.31034
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.24189
[3]	valid_0's multi_logloss: 1.18161
[4]	valid_0's multi_logloss: 1.1357
[5]	valid_0's multi_logloss: 1.08979
[6]	valid_0's multi_logloss: 1.0541
[7]	valid_0's multi_logloss: 1.01417
[8]	valid_0's multi_logloss: 0.980564
[9]	valid_0's multi_logloss: 0.95256
[10]	valid_0's multi_logloss: 0.920911
[11]	valid_0's multi_logloss: 0.898194
[12]	valid_0's multi_logloss: 0.880716
[13]	valid_0's multi_logloss: 0.856573
[14]	valid_0's multi_logloss: 0.838118
[15]	valid_0's multi_logloss: 0.821115
[16]	valid_0's multi_logloss: 0.803997
[17]	valid_0's multi_logloss: 0.788368
[18]	valid_0's multi_logloss: 0.77776
[19]	valid_0's multi_logloss: 0.766919
[20]	valid_0's multi_logloss: 0.758397
[21]	valid_0's multi_logloss: 0.749307
[22]	valid_0's multi_logloss: 0.739203
[23]	valid_0's multi_logloss: 0.733061
[24]	valid_0's multi_logloss: 0.72484
[25]	valid_0's multi_log

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [14]:
# define and fit 
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
t.append(round(model.score(X_test, y_test)*100, 2))
pickle.dump(model, open('../model/xgb_ng_fs.sav', 'wb'))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 4ms/step - loss: 2.1786 - acc: 0.2225 - val_loss: 1.3061 - val_acc: 0.4149
Epoch 2/100
373/373 [==============================] - 0s 61us/step - loss: 1.9127 - acc: 0.2654 - val_loss: 1.3591 - val_acc: 0.4574
Epoch 3/100
373/373 [==============================] - 0s 89us/step - loss: 1.8433 - acc: 0.3137 - val_loss: 1.5017 - val_acc: 0.4894
Epoch 4/100
373/373 [==============================] - 0s 71us/step - loss: 1.6841 - acc: 0.3592 - val_loss: 1.6397 - val_acc: 0.5106
Epoch 5/100
373/373 [==============================] - 0s 74us/step - loss: 1.6561 - acc: 0.3753 - val_loss: 1.6861 - val_acc: 0.5000
Epoch 6/100
373/373 [==============================] - 0s 71us/step - loss: 1.7015 - acc: 0.3700 - val_loss: 1.6637 - val_acc: 0.5000
Epoch 7/100
373/373 [==============================] - 0s 53us/step - loss: 1.6823 - acc: 0.3753 - val_loss: 1.5997 - val_acc: 0.5000
Epoch 8/100
373/37

In [16]:
f.append(t)

In [17]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index=['Feature_Selection_Positive', 'Feature_Selection_Negative']
)

,Random Forest,LightGBM,XGBoost,Keras
Feature_Selection_Positive,71.43,84.52,81.25,61.61
Feature_Selection_Negative,70.21,72.34,73.40,61.70


In [18]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index=['Feature_Selection_Positive', 'Feature_Selection_Negative']
).to_csv('../result/Feature_Selection_result.csv')